In [1]:
#install the required packages
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install keras==2.1.6

In [3]:
#import the required items

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.layers.convolutional import Conv2D # to add convolutional layers
from keras.layers.convolutional import MaxPooling2D # to add pooling layers
from keras.layers import Flatten # to flatten data for fully connected layers

import pandas as pd
import numpy as np

In [4]:
# import the data

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

#how many data points do we have?
concrete_data.shape

(1030, 9)

In [5]:
#check for missing data values

concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum() #clean the data

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
#split data into predictors and target

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [8]:
#sanity check
predictors.head()
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
#normalize the data
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
#save the number of predictors to n_cols to build our network
n_cols = predictors_norm.shape[1] # number of predictors

In [11]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) 
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu')) #3 hidden layers, 10 nodes each
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error') #use the adam optimizer and mean squared error as the loss function. Also track the MSE
    return model

In [12]:
# build the model
model = regression_model()

In [13]:
# fit the model
history = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2) 



Train on 721 samples, validate on 309 samples
Epoch 1/50


2024-02-07 00:18:25.615103: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-02-07 00:18:25.622524: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394300000 Hz
2024-02-07 00:18:25.623164: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ce3a470660 executing computations on platform Host. Devices:
2024-02-07 00:18:25.623216: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-02-07 00:18:25.734483: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 1654.3708 - val_loss: 1189.7676
Epoch 2/50
 - 0s - loss: 1624.0531 - val_loss: 1167.3023
Epoch 3/50
 - 0s - loss: 1585.2812 - val_loss: 1138.7577
Epoch 4/50
 - 0s - loss: 1533.6960 - val_loss: 1100.5829
Epoch 5/50
 - 0s - loss: 1462.5079 - val_loss: 1049.8440
Epoch 6/50
 - 0s - loss: 1366.3229 - val_loss: 981.8752
Epoch 7/50
 - 0s - loss: 1239.4315 - val_loss: 893.1311
Epoch 8/50
 - 0s - loss: 1077.3832 - val_loss: 786.3622
Epoch 9/50
 - 0s - loss: 889.5209 - val_loss: 668.6881
Epoch 10/50
 - 0s - loss: 694.5315 - val_loss: 549.1227
Epoch 11/50
 - 0s - loss: 523.4714 - val_loss: 444.2492
Epoch 12/50
 - 0s - loss: 400.9586 - val_loss: 360.5234
Epoch 13/50
 - 0s - loss: 326.9161 - val_loss: 305.6178
Epoch 14/50
 - 0s - loss: 287.9345 - val_loss: 266.3072
Epoch 15/50
 - 0s - loss: 262.9731 - val_loss: 242.7158
Epoch 16/50
 - 0s - loss: 248.0868 - val_loss: 224.7647
Epoch 17/50
 - 0s - loss: 236.5380 - val_loss: 215.7153
Epoch 18/50
 - 0s - loss: 228.4064 - val_loss: 205.6027

In [14]:
#run the model 50 times

# Lists to store final MSE for each run
mse_list = []

for i in range(50):  # Loop to run the model 50 times
    
    # Create and compile the model
    model = regression_model()
    
    # Fit the model
    history = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2) 
    
    # Retrieve the last MSE value
    mse_validation = history.history['val_loss'][-1]
    
    # Append the final MSEs to the list
    mse_list.append(mse_validation)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 1687.0898 - val_loss: 1207.1714
Epoch 2/50
 - 0s - loss: 1652.1093 - val_loss: 1175.6312
Epoch 3/50
 - 0s - loss: 1596.9068 - val_loss: 1125.0170
Epoch 4/50
 - 0s - loss: 1502.1906 - val_loss: 1042.1760
Epoch 5/50
 - 0s - loss: 1346.2871 - val_loss: 914.2493
Epoch 6/50
 - 0s - loss: 1109.4807 - val_loss: 741.6803
Epoch 7/50
 - 0s - loss: 823.5899 - val_loss: 551.1650
Epoch 8/50
 - 0s - loss: 542.5442 - val_loss: 396.7034
Epoch 9/50
 - 0s - loss: 360.0387 - val_loss: 296.8583
Epoch 10/50
 - 0s - loss: 280.4892 - val_loss: 244.1538
Epoch 11/50
 - 0s - loss: 246.1427 - val_loss: 216.5878
Epoch 12/50
 - 0s - loss: 228.5264 - val_loss: 201.3038
Epoch 13/50
 - 0s - loss: 218.5640 - val_loss: 192.4775
Epoch 14/50
 - 0s - loss: 211.2133 - val_loss: 186.5093
Epoch 15/50
 - 0s - loss: 205.7087 - val_loss: 183.3576
Epoch 16/50
 - 0s - loss: 199.6414 - val_loss: 178.9054
Epoch 17/50
 - 0s - loss: 194.8726 - val_loss: 173.8573
E

In [15]:
#print the list

print(mse_list)

[130.38986823319618, 138.5155785322961, 132.82955784473603, 154.19047605644153, 139.93474741815362, 159.88860224751593, 139.1725872743477, 133.12317659939762, 175.08928919918714, 128.19331345048923, 134.42405231562248, 144.56807158139918, 146.3104011017142, 131.4977437732289, 161.9918448438922, 131.85101565265347, 125.49419442581127, 174.83680907808076, 169.26565420897649, 120.4065274976218, 157.13696590287785, 131.2238609783086, 120.26587323926414, 129.49585109081085, 150.88140345699964, 142.6166858858275, 133.33244699027546, 129.8522763051647, 169.52950958449478, 118.49545917696166, 136.66679648982668, 125.64174516301325, 120.04655597279373, 160.12572210120538, 151.93682431711733, 176.1587685396756, 150.2127140378489, 126.15819734740026, 111.42333643644758, 132.11007813882674, 114.4194080636725, 118.39635400864684, 161.67671598662835, 157.02787203001745, 149.58525041006143, 130.7624092225504, 98.18263427339325, 114.36257638283146, 170.1367460084193, 120.88384017203619]


In [16]:
#calculate the mean and standard deviation of the list

import statistics

# Calculate mean
mean_value = statistics.mean(mse_list)
print(mean_value)

# Calculate standard deviation
std_dev = statistics.stdev(mse_list)
print(std_dev)

139.6143677809632
18.840427396685755
